In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch
import kornia
import numpy as np
import gc
import os
import sys
%matplotlib notebook
import matplotlib.pyplot as plt

pwd = os.path.abspath(os.path.join('.'))
pwd = pwd.split('/')
pwd = pwd[0:-1]
pwd.append('src')

module_path = '/'.join(pwd)
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import Trainer
from helpers import Logger
from helpers import Trial
from mibi_dataloader import MIBIDataLoader
from modules import SelfSupervisedEstimator
from criteria import SelfSupervisedEstimatorLoss
import utils

In [2]:
data_dir = '/Users/raymondbaranski/Desktop/data/'
main_folder = '/Users/raymondbaranski/Desktop/data/'

label_dict = dict()
label_dict['A'] = 0
label_dict['B'] = 1

label_args = dict()
label_args['label_type'] = 'categorical'
label_args['label_format'] = 'csv'
label_args['csv_path'] = main_folder + 'mat_files/labels.csv'
label_args['label_dict'] = label_dict

dataset = MIBIDataLoader(main_folder + 'mat_files', 'mat_file', 'marker', label_args, markers=['H3', 'HLADR', 'HLAG'], field='lambdas')
dataset.rename_field('samples', 'x')
dataset.rename_field('labels_onehot', 'y')
dataset.set_return_fields(['x'])
dataset.set_crop(32)
dataset.set_stride(32)

In [3]:
main_dir = '/Users/GitHub/Denoisotron/'
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

estimator_args = dict()
estimator_args['kdims'] =  [[3, 9, 3], [4, 16, 4], [0, 1, 0]]
estimator_args['nfilts'] = [0]

torch.cuda.empty_cache()
estimator = SelfSupervisedEstimator(**estimator_args)
# estimator = SelfSupervisedEstimator.load_model(main_dir + 'junk/', 'amazing_model')

estimator_logger = Logger(['loss'])
estimator_trainer = Trainer()

In [4]:
# Estimator training parameters
estimator_train_args = dict()
estimator_train_args['lr'] = 0.0001
estimator_train_args['batch_size'] = 100
estimator_train_args['epochs'] = 10
estimator_train_args['report'] = 5
estimator_train_args['crop'] = 121
estimator_train_args['clip'] = 1
estimator_train_args['decay'] = 0
estimator_train_args['restart'] = True
estimator_train_args['epoch_frac'] = 1
# estimator_train_args['error_target'] = 0.4
# estimator_train_args['decay'] = 1e-5

# Estimator loss parameters
estimator_criterion = SelfSupervisedEstimatorLoss()

dataset.set_crop(estimator_train_args['crop'])

# estimator.noise = torch.tensor(1)

In [6]:
estimator.to(device)
estimator_train_args['continue'] = True
estimator_trainer.train(estimator, dataset, estimator_criterion, estimator_logger, data_dir + 'junk/', **estimator_train_args)
# print(estimator.noise)

    Minibatch:14/50 > < loss: 2.02517,                                                                                                     

KeyboardInterrupt: 